# Análisis comparativo empresarial frente al índice S&P 500

Este trabajo tiene como objetivo el estudio del S&P500 e intentar ver si las grandes empresas del índice han tenido mejor evolución en relación a los precios que el índice completo en este último año. Para ello, además de las rentabilidades, hemos obtenido algunas variables más que iremos explicando para luego finalmente ver gráficamente como se ven representadas estas variables.

Por tanto, la hipótesis principal que tendrá este análisis será:

### ***Las 5 empresas más grandes del S&P 500 han conseguido batir al índice en rentabilidad***


In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import sys
sys.path.append('../utils')
import funciones
import importlib
importlib.reload(funciones)
from funciones import *
import yfinance as yf

Lo primero que haremos será, mediante Web Scrapping sacar los nombres, símbolos y sector al que pertenecen las empresas que componen el índice. De esta manera podremos tener una idea más clara de él.

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

response = requests.get(url)
response.raise_for_status()
soup_wikipedia = BeautifulSoup(response.text, "html.parser")
tabla = soup_wikipedia.find_all("table", {"id": "constituents"})
df = pd.read_html(str(tabla))[0]
df_empresas = df[["Symbol","Security","GICS Sector"]]
df_empresas.to_csv("sp500_companies_names", index = False)

Una vez sacados los datos del índice, utilizaremos la libreria yfinance para poder sacar los datos de cada una de las empresas así como del índice. Hemos utilizado solo 5 empresas del índice pero éstas representan el 25% del índice.

In [ ]:
#Incluyo los tickers para sacar luego los dataframes
apple = yf.Ticker("AAPL") #Representa el 6.73% del índice
microsoft = yf.Ticker("MSFT") #Representa el 6.20% del índice
nvidia = yf.Ticker("NVDA") #Representa el 5.68% del indice
amazon = yf.Ticker("AMZN") #Representa el 3.78% del indice
meta = yf.Ticker("META") #Representa el 2.57% del indice
sp500 = yf.Ticker("^GSPC")

df_apple = apple.history(start = "2024-01-01", end = "2025-01-01", interval = "1d")
df_microsoft = microsoft.history(start = "2024-01-01", end = "2025-01-01", interval = "1d")
df_nvidia = nvidia.history(start = "2024-01-01", end = "2025-01-01", interval = "1d")
df_amazon = amazon.history(start = "2024-01-01", end = "2025-01-01", interval = "1d")
df_meta = meta.history(start = "2024-01-01", end = "2025-01-01", interval = "1d")
df_sp500 = sp500.history(start = "2024-01-01", end = "2025-01-01", interval = "1d")


#Una vez sacados los dataframes, para poder guardarlos como excels,
#como tienen un formato de fecha con zona horaria y esto no lo reconoce excel, tenemos que cambiar el tipo de fecha:
for df in [df_apple, df_microsoft, df_nvidia, df_amazon, df_meta, df_sp500]:
    df.reset_index(inplace=True)
    df["Date"] = pd.to_datetime(df["Date"]).dt.tz_localize(None)

#Ahora podemos guardar los datos por separado, cada uno en un excel que podremos ver en la carpeta data.
df_apple.to_excel("../data/historico_apple.xlsx", index=False)
df_microsoft.to_excel("../data/historico_microsoft.xlsx", index=False)
df_nvidia.to_excel("../data/historico_nvidia.xlsx", index=False)
df_amazon.to_excel("../data/historico_amazon.xlsx", index=False)
df_meta.to_excel("../data/historico_meta.xlsx", index=False)
df_sp500.to_excel("../data/historico_sp500.xlsx", index=False)



Haremos ahora un paréntesis para explicar paso a paso las funciones utilizadas para finalmente crear unas tablas con los datos finales:

In [ ]:
#funcion para obtener la rentabilidad diaria de cada accion tomando en cuenta el precio de cierre

def rentabilidad_diaria(df, columna_cierre = "Close"):
    df["rentabilidad_diaria"] = df[columna_cierre].pct_change()
    return df.dropna()

# Dentro del df tenemos distintas columnas de precios, pero utilizamos el de cierre porque es el que marca el precio fijo del día.

#Funcion para obtener la rentabilidad anual:
def rentabilidad_anual(df, columna_rentabilidad="rentabilidad_diaria"):
    numero_dias = len(df)
    rentabilidad_anualizada = ((1 + df[columna_rentabilidad]).prod() ** (252 / numero_dias) - 1)*100
    return round(rentabilidad_anualizada,2)

#Para poder calcular la rentabilidad anual, sumamos 1 a cada rentabilidad diaria y las multiplicamos todas
#Para poder conseguir el rendimiento total acumulado de todos los dias
#Lo elevamos a esa fraccion para anualizar la rentabilidad utilizando 252 como númuro usual de dias hábiles en el mercado
#bursátil entre los días totales recopilados.
#Finalmente le restamos 1 y lo multiplicamos *100 para poder cambiar ese crecimiento en formato de rentabilidad.

#Funcion para calcular la volatilidad anualizada que ha tenido cada empresa y el índice. 

def volatilidad_accion_anualizada(df, columna_rentabilidad="rentabilidad_diaria"):
    volatilidad = df[columna_rentabilidad].std() * np.sqrt(252)
    return round(volatilidad,2)


#Calculamos el ratio de Sharpe porque es una variable bastante importante de cara a estudiar valores de renta variable.

def ratio_sharpe(df, columna_rentabilidad="rentabilidad_diaria", rf = 0.04): # La rf corresponde con la tasa libre de riesgo. Hemos tomado una tasa 
#de 4% que es aproximadamente la que ha habido durante este año.
    rent_anual = rentabilidad_anual(df, columna_rentabilidad) / 100
    volatilidad = volatilidad_accion_anualizada(df,columna_rentabilidad)
    sharpe =round((rent_anual-rf)/volatilidad,2)
    return sharpe

#Como el ratio de sharpe se utiliza para las acciones, necesitamos otra variable para relacionar
#estas acciones con el índice. Para ello utilizaremos la beta.

#La beta se utiliza para medir la volatilidad que ha tenido en este caso, las acciones frente al índice al que pertenecen.
#a mayor beta, mayor sensibiliad tiene a las variaciones la acción frente a su índice en este caso.

def beta_accion(df_accion,df_indice,columna_rentabilidad="rentabilidad_diaria"):
    returns_accion = df_accion[columna_rentabilidad].pct_change().dropna()
    returns_indice = df_indice[columna_rentabilidad].pct_change().dropna()

    covarianza = returns_accion.cov(returns_indice)
    varianza_indice = returns_indice.var()
    beta = round(covarianza/varianza_indice, 2)
    return beta

#Por último, utilizaremos la siguiente funcion para recopilar todos estos datos sacados:
def detalle_accion(df, columna_rentabilidad="rentabilidad_diaria", nombre_accion="Acción", df_indice=None, columna_cierre="Close", rf=0.04):
    rent_anual = rentabilidad_anual(df, columna_rentabilidad)
    volatilidad = volatilidad_accion_anualizada(df, columna_rentabilidad)
    sharpe = ratio_sharpe(df, columna_rentabilidad, rf)
    
    if df_indice is not None:
        beta = beta_accion(df, df_indice, columna_cierre)
    else:
        beta = None

    resumen = pd.DataFrame({
        "Acción": [nombre_accion],
        "Rentabilidad Anual (%)": [rent_anual],
        "Volatilidad Anual": [volatilidad],
        "Ratio de Sharpe": [sharpe],
        "Beta": [beta]
    })

    return resumen



A continuación sacaremos todos los datos calculados utilizando la última función explicada y los añadiremos a un dataframe común:

In [ ]:
detalle_apple = detalle_accion(df_apple, nombre_accion="Apple", df_indice=df_sp500)
detalle_microsoft = detalle_accion(df_microsoft, nombre_accion="Microsoft", df_indice=df_sp500)
detalle_nvidia = detalle_accion(df_nvidia, nombre_accion="Nvidia", df_indice=df_sp500)
detalle_amazon = detalle_accion(df_amazon, nombre_accion="Amazon", df_indice=df_sp500)
detalle_meta = detalle_accion(df_meta, nombre_accion="Meta", df_indice=df_sp500)
detalle_sp500 = detalle_accion(df_sp500, nombre_accion="S&P 500")

df_datos_detalle = pd.concat([
    detalle_apple,
    detalle_microsoft,
    detalle_nvidia,
    detalle_amazon,
    detalle_meta,
    detalle_sp500
], ignore_index=True)

df_datos_detalle.to_excel("../data/detalle_acciones_SP500.xlsx", index=True)

Una vez tenemos todos los datos en la misma tabla, para poder obtener una conclusión sobre nuestra hipótesis tuvimos que añadir una columna en la que incluyesemos la diferencia entre cada rentabilidad anual de cada acción con el índice para ver si se cumplía.

In [ ]:
df_completo = pd.read_excel("../data/detalle_acciones_SP500.xlsx")

rentabilidad_indice = df_completo.loc[df_completo["Acción"] == "S&P 500", "Rentabilidad Anual (%)"].values[0]

df_completo["Rentabilidad Comparada con el índice"] = df_completo["Rentabilidad Anual (%)"] - rentabilidad_indice

df_completo.to_excel("../data/tabla_comparativa.xlsx", index=False)

Finalmente obtenemos la siguiente tabla:

![Análisis Rentabilidad frente al índice](../img/Análisis%20Rentabilidad%20frente%20al%20índice.png)

Con esta tabla podemos llegar a la conclusion de que la única acción que no ha superado en rentabilidad anual al índice ha sido Microsoft.